In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [2]:
train=pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
train

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [3]:
!pip install keras-core --upgrade
#(-q 출력메세지 안보이게하는것)
!pip install -q keras-nlp --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 14.2 MB/s eta 0:00:00 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.


In [4]:
import os 
os.environ['KERAS_BACKEND'] = 'tensorflow'
import tensorflow as tf
import keras_core as keras
import keras_nlp
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

2024-04-03 02:08:24.802292: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-03 02:08:24.802416: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-03 02:08:24.930659: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Using TensorFlow backend


In [5]:
from sklearn.model_selection import train_test_split
#이미지나 머신러닝은 test_size 는 영향을 조금 받는다. 보통 8 대 2 로 나눈다. 텍스트 문제는 test split 에 따라 점수 영향을 많이 받지않는다. 
x_train,x_valid,y_train,y_valid=train_test_split(train["text"],train["target"],test_size=0.15,random_state=42,stratify=train["target"])


In [6]:
test=pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")
test



,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [7]:
#BERT 에 맞는 전처리 와 모델링이 따로 있다.

#ram 소모 작고 가벼운 BERT 모델

#모델 이름을 넣을 때 규칙이 있음 preset("x(모델이름)_base/large (모델 경량/무거운 버전쓸건지)_en (데이터 언어)_대소문자구별")
#sequence_length 패딩같은느낌

#데이터전처리
preprocessing=keras_nlp.models.DistilBertPreprocessor.from_preset("distil_bert_base_en_uncased",sequence_length=128)


#문장 자르고 단어마다 토큰화 시키고 (+숫자화) 그 단어길이를 패딩화시킨다 (길이를 표준화시킨다/맞춘다)



Attaching 'tokenizer.json' from model 'keras/distil_bert/keras/distil_bert_base_en_uncased/2' to your Kaggle notebook...
Attaching 'tokenizer.json' from model 'keras/distil_bert/keras/distil_bert_base_en_uncased/2' to your Kaggle notebook...
Attaching 'assets/tokenizer/vocabulary.txt' from model 'keras/distil_bert/keras/distil_bert_base_en_uncased/2' to your Kaggle notebook...


In [15]:
#모델링
model=keras_nlp.models.DistilBertClassifier.from_preset("distil_bert_base_en_uncased",preprocessor=preprocessing,num_classes=2)


Attaching 'config.json' from model 'keras/distil_bert/keras/distil_bert_base_en_uncased/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/distil_bert/keras/distil_bert_base_en_uncased/2' to your Kaggle notebook...
Attaching 'model.weights.h5' from model 'keras/distil_bert/keras/distil_bert_base_en_uncased/2' to your Kaggle notebook...
/opt/conda/lib/python3.10/site-packages/keras_nlp/src/models/backbone.py:46: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids
/opt/conda/lib/python3.10/site-packages/keras_nlp/src/models/backbone.py:46: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids


In [23]:
#sparse 를 적는 이유는 우리 모델이 정답값을 회귀적으로 푸는것보다 분류 문제로 풀어라고 해주는 것 
#val_loss 가 바뀌지 않는것은 이상하다
#learning_rate 를 잘 설정해야줘야한다

import tensorflow as tf


model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),metrics=['acc'],loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True))
model.fit(x_train,y_train,validation_data=(x_valid,y_valid),epochs=2)


Epoch 1/2
203/203 [==============================] - 114s 472ms/step - loss: 0.4891 - acc: 0.7735 - val_loss: 0.3846 - val_acc: 0.8398
Epoch 2/2
203/203 [==============================] - 89s 439ms/step - loss: 0.3660 - acc: 0.8495 - val_loss: 0.3785 - val_acc: 0.8494


In [24]:
#텍스트 문제 풀때는 text 라는 column 하나에 모든 정보를 합쳐서 넣어야한다. (train 똑같이 그렇게 해야한다)

#모델링할때 model.add softmax 는 확률로 바꿔준다 
#BERT 는 매우 sensitive 한 모델이기 때문에 hyper-parameter 를 잘 설정해야한다 

result=model.predict(test["text"])
result

102/102 [==============================] - 16s 141ms/step


array([[-0.8068458,  0.7305441],
       [-1.578373 ,  1.5386708],
       [-1.4110578,  1.3712299],
       ...,
       [-2.0540514,  1.8619642],
       [-1.5623642,  1.4639924],
       [-1.2828496,  1.131856 ]], dtype=float32)

In [25]:
sub=pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")
sub
#확률로 제출- 평가방식이 logloss/ROC/AUC 
#평가방식이 F-1 이나 accuracy 는 클래스로 제출

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0
...,...,...
3258,10861,0
3259,10865,0
3260,10868,0
3261,10874,0


In [26]:
sub['target']=result.argmax(1)
sub

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1


In [27]:
sub.to_csv("sub123.csv",index=0)

In [ ]:
##https://www.kaggle.com/competitions/nlp-getting-started